In [1]:
#import needed librar
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import json
import psycopg2
import os
import re
# 

In [2]:
def get_connection():
    conn = {}
    with open('../db_connection/db_config_2.json') as my_file:
        db_conf = json.load(my_file)
    conn_str = "host={} dbname={} user={} password={}".format(
                db_conf['host'], db_conf['database'], 
                db_conf['user'], db_conf['passw'])
    try:
        conn = psycopg2.connect(conn_str)
        return conn
    except:
        return 'error in connecting to database'

In [3]:
get_connection()

<connection object at 0x7f67a949a668; dsn: 'host=41.89.93.180 password=xxx user=dsl dbname=mohdsl', closed: 0>

In [4]:
def get_products():
    ''' Get all kemsa products '''
    conn = get_connection()
    
    all_products = pd.read_sql('SELECT DISTINCT(m_product_id), product, dateordered FROM fact_kemsa_order', conn)
    
    return all_products

In [5]:
pd.read_sql('SELECT m_product_id, product, dateordered FROM fact_kemsa_order', get_connection())

,m_product_id,product,dateordered


In [6]:
get_products()


,m_product_id,product,dateordered


In [7]:
def get_year_month_product_orders(year,month):
    ''' Get all kemsa products '''
    conn = get_connection()

    query = ''' SELECT *
                FROM fact_kemsa_order
                WHERE date_part('year',  dateordered ) = %s
                AND date_part('month',  dateordered ) = %s '''%(year,month)
    all_products = pd.read_sql(query, conn)

    return all_products.to_json(orient='records')

In [8]:
get_year_month_product_orders(2014,10)

'[]'

In [9]:
# get_year_month_product_orders(1,1).columns
# ps = get_year_month_product_orders(2015,10)
# ps['dateordered'] = pd.to_datetime(ps['dateordered'])
# ps['year'] = ps.apply(lambda row: row.dateordered.year, axis=1)
# ps[['year']]
# ps

In [10]:
# ps[ps['movementdate'] == '0180-10-01']

In [9]:
def get_facility_year_products(facility_id,year):
    ''' Return a list of all products that have been ordered by the facility'''
    query = """ SELECT  a.m_product_id as id, a.product as name , a.dateordered, a.ordernumber, 
                        a.movementdate, a.movementqty as quantity, a.unitprice, a.qtyordered as quantity_ordered, a.periodid, 
                        b.id as facility_id, b.name as facility_name
                FROM fact_kemsa_order a, facilities_facility b
                WHERE a.facilitycode = b.code
                AND   b.id = '%s' 
                AND date_part('year',  a.dateordered ) = %s"""%(facility_id,year)
    facility_products =  pd.read_sql(query,get_connection())
    
    return facility_products

In [10]:
drugs = get_facility_year_products('2e21b2de-7c5b-4db7-bb1d-26930af9274d', 2015)
drugs['month'] = drugs.apply(lambda row: row.dateordered.month, axis=1)
# drugs = drugs.groupby([,])
functions = {'quantity': {'quantity_received':  'sum' }, 'unitprice': ['mean'], 'quantity_ordered': {'quantity_ordered' : 'sum'} , 'month': {'month' :(lambda x: x )}, }
drugs = drugs.groupby(['id']).agg(functions)
drugs.swaplevel(i=0,j=1, axis=1)
drugs.columns = drugs.columns.droplevel()
drugs = drugs.reset_index()
# drugs.to_json(orient='records')
drugs

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [11]:
drugs = get_facility_year_products('2e21b2de-7c5b-4db7-bb1d-26930af9274d', 2015)
drugs['month'] = drugs.apply(lambda row: row.dateordered.month, axis=1)
# drugs = drugs.groupby([,])
functions = {'quantity':['sum'], 'unitprice': ['mean'], 'quantity_ordered':'sum'}
drugs = drugs.groupby(['month','id', 'name']).agg(functions)
drugs.swaplevel(i=0,j=1, axis=1)
drugs.columns = drugs.columns.droplevel()
drugs

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [ ]:
pd.read_sql( ('''SELECT * FROM facilities_facility WHERE code = '%s' ''')%(13425), get_connection() )

In [ ]:
query = """ SELECT  a.m_product_id as id, a.product as name , 
                        SUM(a.movementqty) as quantity, SUM(a.qtyordered) as quantity_ordered, 
                        AVG(unitprice) as average_price, date_part('month',  a.dateordered ) as month
                FROM fact_kemsa_order a, facilities_facility b
                WHERE a.facilitycode = b.code
                AND   b.id = '%s' 
                AND date_part('year',  a.dateordered  ) = %s
                GROUP BY a.m_product_id , a.product, a.dateordered"""%('2e21b2de-7c5b-4db7-bb1d-26930af9274d', 2015)

pd.read_sql(query, get_connection())

In [ ]:
query = """ SELECT DISTINCT(facilitycode) FROM fact_kemsa_order """
pd.read_sql(query, get_connection())